In [ ]:
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
from joblib import load, dump, Parallel, delayed

from scipy.stats import pearsonr
from sklearn.model_selection import GroupKFold
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Lasso

# load data

In [ ]:
data_path = "../inputs/ubiquant-market-prediction"
train = pd.read_pickle(os.path.join(data_path, "train_demodel.pkl"))

# oof

In [ ]:
oof_arr = [0] * len(train)

In [ ]:
kfold = GroupKFold(n_splits=4)

for fold_id, (trn_idx, val_idx) in enumerate(kfold.split(train, train["target"], train["time_id"])):
    
    pred = load("../ckpts/cnn_seed_2022/fold-{}-ohem-1-CyclicLR-lr-0.004-hidden-64-dropout-0.15-l2-0.02-l1-0.0002-epoch-15-direction-1/val_preds_fold_{}.pkl".format(fold_id, fold_id))
    
    for pred_idx, idx in enumerate(val_idx):
        oof_arr[idx] = pred[pred_idx]

In [ ]:
train["oof_cnn"] = np.array(oof_arr)

In [ ]:
train["oof_cnn"].head()

In [ ]:
train["target"].head()

In [ ]:
corr, _ = pearsonr(train["oof_cnn"].values.flatten(), train["target"].values.flatten())

print(corr)

In [ ]:
avg_corr = []

for time_id in tqdm(sorted(list(dict.fromkeys(train["time_id"])))):
    
    oof_by_time_id = train.loc[train["time_id"] == time_id, "oof_cnn"].values.flatten()
    target_by_time_id = train.loc[train["time_id"] == time_id, "target"].values.flatten()
    
    corr_by_time_id, _ = pearsonr(oof_by_time_id, target_by_time_id)
    avg_corr.append(corr_by_time_id)
    
print(np.mean(avg_corr))

# lasso

In [ ]:
lasso = Lasso(alpha=0.1).fit(X=np.expand_dims(train["oof_cnn"].values, axis=1) * 5, y=train["target"].values)

In [ ]:
lasso_pred = lasso.predict(np.expand_dims(train["oof_cnn"].values, axis=1))

In [ ]:
lasso_pred

In [ ]:
corr, _ = pearsonr(train["oof_cnn"].values, train["target"].values - 10 * lasso_pred)
print(corr)

corr, _ = pearsonr(train["target"].values, train["target"].values - 10 * lasso_pred)
print(corr)

In [ ]:
train["target_demodel_cnn"] = train["target"].values - 10 * lasso_pred

In [ ]:
col = "target_demodel_cnn"
col_mean = train[[col, "time_id"]].groupby("time_id").mean().rename(columns={col : "{}_mean".format(col)})
col_std = train[[col, "time_id"]].groupby("time_id").std().rename(columns={col : "{}_std".format(col)})

train = train.merge(col_mean, on="time_id")
train = train.merge(col_std, on="time_id")

train[col] = (train[col] - train["{}_mean".format(col)]) / train["{}_std".format(col)]

train = train.drop(["{}_mean".format(col), "{}_std".format(col)], axis=1)

In [ ]:
demodel_scaler = StandardScaler().fit(np.expand_dims(train["target_demodel_cnn"].values, axis=1))

train_demodel_mean, train_demodel_var = demodel_scaler.mean_, demodel_scaler.var_

train["target_demodel_cnn"] = train["target_demodel_cnn"].clip(
            train_demodel_mean[0] - 3 * np.sqrt(train_demodel_var[0]), 
            train_demodel_mean[0] + 3 * np.sqrt(train_demodel_var[0])
        )

In [ ]:
corr, _ = pearsonr(train["target"].values, train["target_demodel_cnn"].values)
print(corr)

In [ ]:
train.to_pickle(os.path.join(data_path, "train_demodel_cnn.pkl"))